<a href="https://colab.research.google.com/github/EslamAsfour/AUV_Tests_EfficientDet/blob/main/K_Mean_Anchors_Ratios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clone the Github Repo

In [1]:
! git clone https://github.com/mnslarcher/kmeans-anchors-ratios.git

Cloning into 'kmeans-anchors-ratios'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 103 (delta 61), reused 65 (delta 30), pack-reused 0
Receiving objects: 100% (103/103), 32.70 KiB | 6.54 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [2]:
cd kmeans-anchors-ratios/

/content/kmeans-anchors-ratios


In [4]:
import json
import numpy as np
import os

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!pip install -r requirements.txt

#GDrive Mount

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Copy Dataset to the Repo Folder

In [10]:
cp /content/gdrive/MyDrive/Dataset_All_Classes.rar /content/kmeans-anchors-ratios

In [13]:
mkdir dataset

In [ ]:
!unrar x  "/content/kmeans-anchors-ratios/Dataset_All_Classes.rar" "dataset/"

#Path To the Training Annotations JSON file

In [15]:
INSTANCES_PATH = "/content/kmeans-anchors-ratios/dataset/Dataset_All_Classes/annotations/annotations_Train.json"  

In [16]:
with open(INSTANCES_PATH) as f:
    instances = json.load(f)

#Get optimal anchors ratios

In [18]:
## change the following parameters according to your model:

# EfficientDetD{PHI}
PHI = 0  # for another efficientdet change only this, e.g. PHI = 3 for D3


#------------ Added 416 --------------------------- #
input_sizes = [416 ,512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
pyramid_levels = [5, 5, 5, 5, 5, 5, 5, 5, 6]
anchor_scale = [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0]

scale = anchor_scale[PHI]
strides = 2 ** np.arange(3, pyramid_levels[PHI] + 3)
scales = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

INPUT_SIZE = input_sizes[PHI]
ANCHORS_SIZES = (scale * scales * strides[:, np.newaxis]).flatten().tolist()
ANCHORS_SIZES

[32.0,
 40.31747359663594,
 50.79683366298238,
 64.0,
 80.63494719327188,
 101.59366732596476,
 128.0,
 161.26989438654377,
 203.18733465192952,
 256.0,
 322.53978877308754,
 406.37466930385904,
 512.0,
 645.0795775461751,
 812.7493386077181]

In [20]:
from kmeans_anchors_ratios import get_optimal_anchors_ratios

anchors_ratios = get_optimal_anchors_ratios(
    instances,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    normalizes_bboxes=True,
    num_runs=3,
    num_anchors_ratios=3,
    max_iter=300,
    iou_threshold=0.5,
    min_size=0,
    decimals=1,
    default_anchors_ratios=[(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
)

[12/17 20:01:14] Starting the calculation of the optimal anchors ratios
[12/17 20:01:14] Extracting and preprocessing bounding boxes
[12/17 20:01:14] Discarding 3 bounding boxes with size lower or equal to 0


[12/17 20:01:14] K-Means (3 runs): 100%|██████████████████| 3/3 [00:00<00:00, 65.19it/s]

	Runs avg. IoU: 86.52% ± 0.00% (mean ± std. dev. of 3 runs, 0 skipped)
	Avg. IoU between bboxes and their most similar anchors after norm. them to make their area equal (only ratios matter): 86.52%
[12/17 20:01:14] Default anchors ratios: [(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
	Avg. IoU between bboxes and their most similar default anchors, no norm. (both ratios and sizes matter): 58.62%
	Num. bboxes without similar default anchors (IoU < 0.5):  2364/5542 (42.66%)
[12/17 20:01:15] K-Means anchors ratios: [(0.3, 2.9), (0.7, 1.3), (1.0, 1.0)]
	Avg. IoU between bboxes and their most similar K-Means anchors, no norm. (both ratios and sizes matter): 67.24%
	Num. bboxes without similar K-Means anchors (IoU < 0.5):  1356/5542 (24.47%)
[12/17 20:01:15] K-Means anchors have an IoU < 50% with bboxes in 18.19% less cases than the default anchors, you should consider to use them


#Generate anchors given ratios and sizes

In [21]:
from kmeans_anchors_ratios import generate_anchors_given_ratios_and_sizes


anchors = generate_anchors_given_ratios_and_sizes(anchors_ratios, ANCHORS_SIZES)
print("Anchors:")
print(anchors)

Anchors:
[[   9.6          92.8       ]
 [  22.4          41.6       ]
 [  32.           32.        ]
 [  12.09524208  116.92067343]
 [  28.22223152   52.41271568]
 [  40.3174736    40.3174736 ]
 [  15.2390501   147.31081762]
 [  35.55778356   66.03588376]
 [  50.79683366   50.79683366]
 [  19.2         185.6       ]
 [  44.8          83.2       ]
 [  64.           64.        ]
 [  24.19048416  233.84134686]
 [  56.44446304  104.82543135]
 [  80.63494719   80.63494719]
 [  30.4781002   294.62163525]
 [  71.11556713  132.07176752]
 [ 101.59366733  101.59366733]
 [  38.4         371.2       ]
 [  89.6         166.4       ]
 [ 128.          128.        ]
 [  48.38096832  467.68269372]
 [ 112.88892607  209.6508627 ]
 [ 161.26989439  161.26989439]
 [  60.9562004   589.24327049]
 [ 142.23113426  264.14353505]
 [ 203.18733465  203.18733465]
 [  76.8         742.4       ]
 [ 179.2         332.8       ]
 [ 256.          256.        ]
 [  96.76193663  935.36538744]
 [ 225.77785214  419.30172541]

#Get bounding boxes adapted to the input size

In [22]:
from kmeans_anchors_ratios import get_bboxes_adapted_to_input_size


resized_bboxes = get_bboxes_adapted_to_input_size(instances, input_size=INPUT_SIZE)
resized_bboxes = resized_bboxes[resized_bboxes.prod(axis=1) > 0]  # remove 0 size
print("Bounding boxes adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes adapted to the input size (first 5):
[[ 44.   73.5]
 [ 81.5 158. ]
 [145.5 146. ]
 [ 16.   29. ]
 [ 43.   67. ]]


#Get the avg. IoU between the bounding boxes and their closest anchors

In [23]:
from kmeans_anchors_ratios import average_iou

avg_iou = average_iou(resized_bboxes, anchors)
print(f"Avg. IoU: {100 * avg_iou:.2f}%")

Avg. IoU: 67.24%


#Get annotations whose bounding boxes don't have similar anchors

In [24]:
from kmeans_anchors_ratios import get_annotations_without_similar_anchors


annotations = get_annotations_without_similar_anchors(
    instances,
    anchors_ratios,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    iou_threshold=0.5,
    min_size=0,
)

bboxes = [ann["bbox"][-2:] for ann in annotations]  # widths and heights
print("Bounding boxes without similar anchors (first 5):")
print(bboxes[:5])

instances_without_similar_anchors = instances.copy()
instances_without_similar_anchors["annotations"] = annotations
resized_bboxes = get_bboxes_adapted_to_input_size(instances_without_similar_anchors, input_size=INPUT_SIZE)
print("Bounding boxes without similar anchors adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes without similar anchors (first 5):
[[16, 29], [14.5, 23], [13, 28], [7.5, 13], [10, 19]]
Bounding boxes without similar anchors adapted to the input size (first 5):
[[16.  29. ]
 [14.5 23. ]
 [13.  28. ]
 [ 7.5 13. ]
 [10.  19. ]]
